In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../Data/housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [2]:
from Linear_regression import LinearRegression

X = df[['households']]
y = df['population']

model = LinearRegression()
model.fit(X, y)
model.SSE(), model.RMSE(), model.var(), model.std()

(np.float64(4683613936.333832),
 np.float64(476.36045161320357),
 np.float64(226941.2702943033),
 np.float64(476.3835327698716))

In [3]:
X_small = np.array([1,2,3,4,5]).reshape(-1,1)
y_small = np.array([2,4,5,4,5])

model = LinearRegression()
model.fit(X_small, y_small)

print("SSE:", model.SSE())
print("MSE:", model.MSE())
print("RMSE:", model.RMSE())
print("std:", model.std())
print("diff std - RMSE:", model.std() - model.RMSE())

SSE: 2.4
MSE: 0.48
RMSE: 0.6928203230275509
std: 0.8944271909999159
diff std - RMSE: 0.20160686797236493


In [4]:
model.fit(X, y)            # se till att modellen är fit:ad
F, p = model.significance()
print(f"F = {F:.4f}, p = {p:.4e}")

alpha = 0.05
if p < alpha:
    print("Modellen är signifikant: förkasta H0 (minst en koefficient ≠ 0).")
else:
    print("Ingen signifikans: kan inte förkasta H0 (inga bevis att någon koefficient ≠ 0).")

F = 95995.2934, p = 0.0000e+00
Modellen är signifikant: förkasta H0 (minst en koefficient ≠ 0).
